# 1.1 토양 경사도 데이터 분석 

In [36]:
import geopandas as gpd
import shapefile 
import glob, json, os, sys, re
from pyproj import Proj, Transformer, transform
from shapely.geometry import Polygon
import pandas as pd
import numpy as np
import mysql.connector
import matplotlib.pyplot as plt
import glob
from tqdm import notebook.tqdm
# sys.path.append('./module') # 한번만

from module.building_polygon import shapeconverter


In [39]:
with open('./path.json') as file:
    json_obj = json.load(file)
    
con = mysql.connector.connect(host = json_obj['DB']['Host'],
                            port = json_obj['DB']['Port'],
                            database = 'capstone',
                            user = json_obj['DB']['User'],
                            password = str(json_obj['DB']['Password'])
                            )

In [40]:
# 부동산 빅데이터 플랫폼에서 제공하는 토지 경사도(1M급) land_slope_1m -> ls1 #
# nsdi 국가공간정보포털에서 제공하는 토지 경사도 -> nsdi_land_slope -> nls # 좌표계 5186
converter = shapeconverter.ShapeConverter(from_=5186)
cursor = con.cursor()
for shp in glob.glob("./data/토지경사도/ES1005AA00101YY2200_zip/LOT_ACCTO_GRDNT_ONE_METER_GRADE_INFO.shp")
    shape_ = shapefile.Reader(ls1,encoding='cp949')
    for shapeRec in notebook.tqdm(shape_.shapeRecords()):
            shape = shapeRec.shape
            record = shapeRec.record
            points = shape.points
            
            try:
                latitude, longitude = converter.get_center(shape)
            except:
                continue

            centerPoint = 'POINT({} {})'.format(latitude, longitude)

            p = converter.convert_parts(converter.get_parts(shapeRec.shape))
            polygontext = converter.get_polygon_text(p)
    
# nls = gpd.read_file("./data/토지경사도/Z_SIS_ASIT_SOILSLOPE_AREA/Z_SIS_ASIT_SOILSLOPE_AREA.shp")


NameError: name 'crs' is not defined

In [28]:
ls1.crs #5186
# 5186 좌표계를 카카오맵 건물 위치와 동일하게 레이어를 겹쳐야함 -> centerpoint 변환 기준을 정확하게 잡아야함

<Derived Projected CRS: EPSG:5186>
Name: Korea 2000 / Central Belt 2010
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore between 126°E and 128°E.
- bounds: (126.0, 33.14, 128.0, 38.33)
Coordinate Operation:
- name: Korea Central Belt 2010
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
ls1.sample(3) # polygon -> centerpoint 변환 한번 해야함 좌표계 뭘로 하는지 못찾겠는데 5174

,PNU,LEGDONG_CD,CTPR_NM,SIGNGU_NM,LEGDONG_NM,LI_NM,LNNO_ADRES,GD_AVE_VL,GD_LWST_VL,GD_TOP_VL,PLOT_DIMS,LOT_CRCMF,geometry
829922,1162010200103780019,1162010200,서울특별시,관악구,신림동,None,378-19답,4.362484,0.226404,7.887933,716.777663,336.766954,"POLYGON ((193803.299 541459.639, 193806.449 54..."
666433,1150010800103720003,1150010800,서울특별시,강서구,공항동,None,372-3 전,18.745795,0.056581,51.100571,3163.401467,280.924302,"POLYGON ((183612.381 549375.390, 183608.105 54..."
614948,1153010700103050007,1153010700,서울특별시,구로구,개봉동,None,305-7대,1.097973,0.634683,1.525337,131.152535,46.185651,"POLYGON ((186943.588 543814.949, 186955.798 54..."


In [25]:
ls1[ls1["CTPR_NM"] == '서울특별시']

,PNU,LEGDONG_CD,CTPR_NM,SIGNGU_NM,LEGDONG_NM,LI_NM,LNNO_ADRES,GD_AVE_VL,GD_LWST_VL,GD_TOP_VL,PLOT_DIMS,LOT_CRCMF,geometry
0,1135010200104970006,1135010200,서울특별시,노원구,월계동,None,497-6답,2.487450,0.677974,10.367235,12.120133,36.527271,"POLYGON ((204979.649 557735.301, 204985.715 55..."
1,1135010200104720018,1135010200,서울특별시,노원구,월계동,None,472-18대,2.697310,0.192324,7.381097,166.693050,52.390403,"POLYGON ((205122.978 557565.662, 205130.375 55..."
2,1135010200104720019,1135010200,서울특별시,노원구,월계동,None,472-19대,2.812627,0.067573,7.217959,169.117316,52.714721,"POLYGON ((205137.846 557549.463, 205126.629 55..."
3,1156013200137410027,1156013200,서울특별시,영등포구,신길동,None,3741-27천,1.603282,0.010718,5.644447,340.965429,144.168338,"POLYGON ((192297.025 544420.993, 192301.617 54..."
4,1156013200137410018,1156013200,서울특별시,영등포구,신길동,None,3741-18대,1.493227,0.141492,5.326812,753.390605,142.435625,"POLYGON ((192310.120 544391.245, 192307.919 54..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
918328,1135010200105020056,1135010200,서울특별시,노원구,월계동,None,502-56대,3.053454,1.351910,5.543773,8.385499,25.306990,"POLYGON ((204952.968 557763.825, 204953.257 55..."
918329,1135010200105020047,1135010200,서울특별시,노원구,월계동,None,502-47대,14.844489,2.772097,23.877407,27.617732,39.891037,"POLYGON ((204960.564 557754.528, 204961.200 55..."
918330,1135010200105020009,1135010200,서울특별시,노원구,월계동,None,502-9대,11.984658,5.439744,14.362223,16.076887,16.478597,"POLYGON ((204965.263 557740.704, 204967.550 55..."
918331,1135010200104970010,1135010200,서울특별시,노원구,월계동,None,497-10천,4.222654,1.285031,11.997252,77.719491,83.544504,"POLYGON ((204963.579 557735.227, 204965.161 55..."


In [8]:
nls.sample(3) #

,AREA,PERIMETER,CODE_SL,SOILSLOPE,geometry
308913,5934.89,454.839,3.0,7-15%,"POLYGON ((422313.406 253261.359, 422344.844 25..."
291768,80726.40,2560.520,4.0,15-30%,"POLYGON ((303674.406 267920.250, 303644.625 26..."
131250,49523.20,1363.660,3.0,7-15%,"POLYGON ((376084.469 372832.812, 376089.781 37..."
